<h1> Sign Language Neural Network </h1>
<p>by Robert Kulesza</p>

## Dataset
* Sign Language MNIST dataset
* https://www.kaggle.com/datamunge/sign-language-mnist#sign_mnist_test.zip

## Methods
* <b>Preprocessing</b>
    * batch processing
* <b>Machine Learning Techniques</b>
    * Neural Network
* <b>Results</b>
    * loss
    * accuracy

### import libraries

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

from sklearn import svm
from sklearn.model_selection import learning_curve

import numpy as np
import random

import os
from os import listdir
from os.path import isfile, join

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import pandas as pd

from sklearn.preprocessing import LabelEncoder

### Load Data

* paths for training and testing data.
* list of csv files in the training / testing directories.
* separated data and labels
* reshaped data to 28*28

In [ ]:
csv_train = "../input/sign-mnist/sign_mnist_train.csv"
csv_test =  "../input/sign-mnist/sign_mnist_test.csv"

In [ ]:
df_train = pd.read_csv(csv_train)
df_test = pd.read_csv(csv_test)

array_train_X = np.array(df_train.drop("label",1))
array_train_y = np.array(df_train['label'])

array_test_X = np.array(df_test.drop("label",1))
array_test_y = np.array(df_test['label'])

In [ ]:
array_train_X = array_train_X.reshape(len(array_train_X),28,28)
array_test_X = array_test_X.reshape(len(array_test_X),28,28)

### print example image and label

In [ ]:
img = array_train_X[0]
lbl = array_train_y[0]

print("label: ",lbl)
print(plt.imshow(img, cmap='gray'))

### Preprocessing

In [ ]:
test_batch_size = int(1*len(array_test_X))
train_batch_size = int(1*len(array_train_X))

In [ ]:
def preprocess(X,y,batch_size):
    indeces = np.array(list(range(0,len(X))))
    random.shuffle(indeces)
    X, y = np.array([X[indeces[a]] for a in range(0,batch_size)]), np.array([y[indeces[a]] for a in range(0,batch_size)])
    X = X.astype(np.float64) / 100
    X = np.tanh(X) #why isn't this working? -- test single values
    #X = 1/(1 + np.exp(-X)) 
    X = X.reshape(batch_size,28,28,1)
    return X,y

In [ ]:
print("test: ",test_batch_size,"/",len(array_test_X))
print("train: ",train_batch_size,"/",len(array_train_X))

print("shape: ",array_train_X.shape)

In [ ]:
array_train_X, array_train_y = preprocess(array_train_X, array_train_y, train_batch_size)
array_test_X, array_test_y = preprocess(array_test_X, array_test_y, test_batch_size)


In [ ]:
print("sample of normalized values: \n\n",array_train_X[0][0])

### Convolutional Neural Network
**create model**

In [ ]:
def Create_Model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28,28,1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    
    model.add(layers.Flatten())
    
    model.add(layers.Dense(400, activation='relu'))
    model.add(layers.Dense(200, activation='tanh'))
    model.add(layers.Dense(26, activation='softmax'))
    model.summary()
    
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    
    return model

def train_test_model(model, X_train, y_train, X_test, y_test):
    history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_test, y_test))
    
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim([0.5, 1])
    plt.legend(loc='lower right')

    test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)
    return history


In [ ]:
model = Create_Model()

In [ ]:
history = train_test_model(model,array_train_X, array_train_y, array_test_X, array_test_y)